In [1]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [2]:
from langchain.document_loaders import PyPDFLoader

# Load PDF
loaders = [
    # Duplicate documents on purpose - messy data
    PyPDFLoader("temp.pdf")
#    PyPDFLoader("docs/cs229_lectures/MachineLearning-Lecture01.pdf"),
#    PyPDFLoader("docs/cs229_lectures/MachineLearning-Lecture02.pdf"),
#    PyPDFLoader("docs/cs229_lectures/MachineLearning-Lecture03.pdf")
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [3]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

In [4]:
splits = text_splitter.split_documents(docs)


In [5]:
len(splits)

27

In [6]:
from langchain_community.embeddings import JinaEmbeddings
#KEY="jina_19d1fe674e6a494eb4339d0b404bb83bpfQ-RUo8zBdNECI0s3ZlWf1bf8Qb"
KEY ="jina_cca4cdb9735f4431a1092347b873d831lOGCSgReRJ5gn0JpGB_PgMqsb2xT"
embedding = JinaEmbeddings(jina_api_key = KEY, model_name="jina-embeddings-v2-base-en")

In [7]:
from langchain.vectorstores import Chroma

In [9]:
persist_directory = 'docs/chroma1/'
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)

In [10]:
print(vectordb._collection.count())

27


In [11]:
question = "is there an email i can ask for help"
docs = vectordb.similarity_search(question,k=3)
print(docs)


[Document(page_content='IMPORTANT NOTICE AND DISCLAIMER\nTI PROVIDES TECHNICAL AND RELIABILITY DATA (INCLUDING DATA SHEETS), DESIGN RESOURCES (INCLUDING REFERENCE \nDESIGNS), APPLICATION OR OTHER DESIGN ADVICE, WEB TOOLS, SAFETY INFORMATION, AND OTHER RESOURCES “AS IS” \nAND WITH ALL FAULTS, AND DISCLAIMS ALL WARRANTIES, EXPRESS AND IMPLIED, INCLUDING WITHOUT LIMITATION ANY \nIMPLIED WARRANTIES OF MERCHANTABILIT Y, FITNESS FOR A PARTICULAR PURPOSE OR NON-INFRINGEMENT OF THIRD \nPARTY INTELLECTUAL PROPERTY RIGHTS.\nThese resources are intended for skilled developers designing with TI products. Y ou are solely responsible for (1) selecting the appropriate \nTI products for your application, (2) designing, validating and testing your application, and (3) ensuring your application meets applicable \nstandards, and any other safety, security, regulatory or other requirements.\nThese resources are subject to change without notice. TI grants you permission to use these resources only for deve

In [12]:
vectordb.persist()

In [13]:
question = "What are major topics for this class?"
docs = vectordb.similarity_search(question,k=3)
len(docs)

3

In [14]:
import datetime
current_date = datetime.datetime.now().date()
if current_date < datetime.date(2023, 9, 2):
    llm_name = "gpt-3.5-turbo-0301"
else:
    llm_name = "gpt-3.5-turbo"
print(llm_name)

gpt-3.5-turbo


In [15]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name=llm_name, temperature=0)

c:\Users\hamza.qureshi\AppData\Local\anaconda3\envs\chatBot\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [16]:
from langchain.chains import RetrievalQA

In [24]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [26]:
question ="what is name mentioned in this?"
result = qa_chain({"query": question})
result["result"]

'The name mentioned in this context is LM741.'

'Based on the provided context, it is not clear what class or subject is being referred to. Therefore, I cannot determine the major topics for this class.'